### Methode :
- **Support Vector Machine (SVM)** : Un algorithme de classification qui trouve l'hyperplan optimal dans un espace de grande dimension pour séparer les différentes classes. Il peut également être étendu pour gérer des problèmes non linéaires en utilisant des noyaux.

- **Régression logistique** : Un algorithme utilisé pour la classification binaire (et pouvant être étendu à la classification multiclasse) en modélisant la probabilité que chaque classe soit la classe cible à l'aide d'une fonction logistique.
- **Random Forest** : Un algorithme d'ensemble utilisé pour la classification et la régression. Il combine les prédictions de plusieurs arbres de décision pour obtenir une prédiction plus robuste et généralement de meilleure qualité.
- **Réseaux de neurones** :
- **Perceptron ou Multi-perceptron** :
- **Gradient Boosting** : Un autre algorithme d'ensemble qui construit des arbres de décision de manière séquentielle, en corrigeant les erreurs des arbres précédents. Cela conduit à un modèle de prédiction puissant.
- **Naive Bayes** : Un classificateur probabiliste simple basé sur le théorème de Bayes avec une forte indépendance entre les fonctionnalités. Il est souvent utilisé pour la classification de texte et d'autres tâches où l'indépendance des fonctionnalités est une hypothèse raisonnable.





In [28]:

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder ,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
import numpy as np




In [29]:
# Load dataset
data_dir_path = '../src/data/'
data_trainset_cleaned = pd.read_csv(data_dir_path + "/data_trainset_cleaned.csv")

In [30]:
# Create a copy to avoid modifying the original DataFrame
dataset_encoded = data_trainset_cleaned.copy()

# Encoding the 'species' column
encoder = LabelEncoder()
dataset_encoded['species'] = encoder.fit_transform(dataset_encoded['species'])

# Select numeric columns except for the already encoded 'species' column for imputation
numeric_columns = dataset_encoded.select_dtypes(include=['int64', 'float64']).columns

# Initialize the KNNImputer instance
imputer = KNNImputer(n_neighbors=5)

# Apply imputation only on the numeric columns
dataset_encoded[numeric_columns] = imputer.fit_transform(dataset_encoded[numeric_columns])

# Convert the NumPy array returned by KNNImputer back into a pandas DataFrame and match column names
trainset_imputed = pd.DataFrame(dataset_encoded, columns=dataset_encoded.columns)

# Calculate the percentage of NaN values for each column after imputation
na_percentage = trainset_imputed.isna().mean() * 100

# Display the percentages
print("Percentage of missing data per column after imputation:")
print(na_percentage)

Percentage of missing data per column after imputation:
species      0.0
margin1      0.0
margin2      0.0
margin3      0.0
margin4      0.0
            ... 
texture60    0.0
texture61    0.0
texture62    0.0
texture63    0.0
texture64    0.0
Length: 187, dtype: float64


# Mixing and Splitting of Data 

In [31]:
# Shuffle the new dataset
new_dataset_shuffled = shuffle(trainset_imputed)

# Separate target from data
features = trainset_imputed.drop("species", axis=1)
new_target = trainset_imputed['species']

# Create training set and testing set
features_train, features_test, target_train, target_test = train_test_split(
    features, new_target, test_size=0.2, stratify=new_target
)

# Standardization of the test set
scaler = StandardScaler()
features_train_standardized = scaler.fit_transform(features_train)
features_test_standardized = scaler.transform(features_test)



# Entrainement

# DecisionTree


In [32]:
# Définir les hyperparamètres à tester
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 10, None]
}

# Créer l'instance de DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

# Configurer GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Exécuter la recherche en grille sur les données d'entraînement standardisées
grid_search.fit(features_train_standardized, target_train)

# Meilleur modèle
best_model = grid_search.best_estimator_

# Prédictions sur l'ensemble de test
predictions = best_model.predict(features_test_standardized)

# Évaluer la performance
accuracy = accuracy_score(target_test, predictions)
f1 = f1_score(target_test, predictions, average='macro')  # ou 'macro' pour une classification multiclasse

print(f"Meilleurs hyperparamètres: {grid_search.best_params_}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Meilleurs hyperparamètres: {'criterion': 'entropy', 'max_depth': 10}
Accuracy: 0.6868686868686869
F1 Score: 0.6619047619047619


In [33]:
# Calculer les scores de validation croisée
cv_scores_accuracy = cross_val_score(best_model, features_train_standardized, target_train, cv=5, scoring='accuracy')
cv_scores_f1 = cross_val_score(best_model, features_train_standardized, target_train, cv=5, scoring='f1_macro')

# Afficher les scores moyens de la validation croisée
print(f"Moyenne des scores de précision de validation croisée : {np.mean(cv_scores_accuracy)}")
print(f"Moyenne des scores F1 de validation croisée : {np.mean(cv_scores_f1)}")

Moyenne des scores de précision de validation croisée : 0.7070137727887906
Moyenne des scores F1 de validation croisée : 0.665993265993266


# LogisticRegression

In [34]:
param_grid_lr = {
    'solver': ['lbfgs', 'liblinear'],
    'C': np.logspace(-3, 3, 7)
}

lr = LogisticRegression(max_iter=1000, penalty='l2', random_state=42)
grid_search_lr = GridSearchCV(estimator=lr, param_grid=param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Assurez-vous d'avoir déjà standardisé features_train et features_test
grid_search_lr.fit(features_train_standardized, target_train)

# Meilleur modèle
best_model_lr = grid_search_lr.best_estimator_

# Prédictions sur l'ensemble de test standardisé
predictions_lr = best_model_lr.predict(features_test_standardized)

# Évaluation de la performance
accuracy_lr = accuracy_score(target_test, predictions_lr)
f1_lr = f1_score(target_test, predictions_lr, average='macro')

print(f"Meilleurs hyperparamètres pour Logistic Regression: {grid_search_lr.best_params_}")
print(f"Accuracy pour Logistic Regression: {accuracy_lr}")
print(f"F1 Score pour Logistic Regression: {f1_lr}")

Fitting 5 folds for each of 14 candidates, totalling 70 fits
Meilleurs hyperparamètres pour Logistic Regression: {'C': 10.0, 'solver': 'lbfgs'}
Accuracy pour Logistic Regression: 0.98989898989899
F1 Score pour Logistic Regression: 0.9865319865319866


In [35]:
# Calculer les scores de validation croisée pour la précision
cv_scores_accuracy_lr = cross_val_score(best_model_lr, features_train_standardized, target_train, cv=5, scoring='accuracy')

# Calculer les scores de validation croisée pour le score F1
cv_scores_f1_lr = cross_val_score(best_model_lr, features_train_standardized, target_train, cv=5, scoring='f1_macro')

# Afficher les scores moyens de la validation croisée
print(f"Moyenne des scores de précision de validation croisée pour Logistic Regression: {np.mean(cv_scores_accuracy_lr)}")
print(f"Moyenne des scores F1 de validation croisée pour Logistic Regression: {np.mean(cv_scores_f1_lr)}")


Moyenne des scores de précision de validation croisée pour Logistic Regression: 0.9987341772151899
Moyenne des scores F1 de validation croisée pour Logistic Regression: 0.9986531986531986
